# Competitor Dashboard: Data Check
**09/13/2017** 
Brainly and Zuoyebang data looks aberrant; there might be many data points that have been updated by SimilarWeb _ex post_. Checked data pipeline on my end; no issues. The following investigation highlights my findings thus far. 

## Hypothesis 1: SimilarWeb changed data _ex post_

My leading hypothesis is that **SimilarWeb has retroactively updated their inferred traffic and engagement stats**, but the reason for this is at present unknown.

In [1]:
# Import dependencies
import pandas as pd

# Show all rows
pd.options.display.max_rows = 5000

# Import raw data
df_new = pd.read_csv('./outfiles/2017-09-13_15:20.csv')
df_old = pd.read_csv('./outfiles/2017-08-15_14:59.csv')

print("Current data shape", df_new.shape)
print("Previous data shape", df_old.shape)

Current data shape (8934, 40)
Previous data shape (8499, 40)


This is a difference I expected since I added Mindspark (in India) to the list of competitors to track in early September. The pipeline is performing the transformations and calculations, so I'm checking _only_ the data provided by the SimilarWeb API.

In [2]:
# Checking which columns to index data by
df_new.columns.values

array(['group_site', 'KA_initiative', 'endpoint_category', 'date',
       'average_visit_duration', 'visits', 'LT_mins', 'norm_LT',
       'average_visit_duration_TTM_sum', 'visits_TTM_sum',
       'LT_mins_TTM_sum', 'norm_LT_TTM_sum',
       'average_visit_duration_TTM_mean', 'visits_TTM_mean',
       'LT_mins_TTM_mean', 'norm_LT_TTM_mean',
       'average_visit_duration_pct_yoy', 'visits_pct_yoy',
       'LT_mins_pct_yoy', 'norm_LT_pct_yoy',
       'average_visit_duration_TTM_sum_pct_yoy', 'visits_TTM_sum_pct_yoy',
       'LT_mins_TTM_sum_pct_yoy', 'norm_LT_TTM_sum_pct_yoy',
       'average_visit_duration_TTM_mean_pct_yoy',
       'visits_TTM_mean_pct_yoy', 'LT_mins_TTM_mean_pct_yoy',
       'norm_LT_TTM_mean_pct_yoy', 'average_visit_duration_pct_mom',
       'visits_pct_mom', 'LT_mins_pct_mom', 'norm_LT_pct_mom',
       'average_visit_duration_TTM_sum_pct_mom', 'visits_TTM_sum_pct_mom',
       'LT_mins_TTM_sum_pct_mom', 'norm_LT_TTM_sum_pct_mom',
       'average_visit_duration_TTM_m

## Validating Hypothesis 1

In [3]:
# Designate columns and indices
cols = ["group_site","KA_initiative","endpoint_category","date","visits","average_visit_duration"]
new_index = cols[0:4]

# Pared down DataFrames
sw_df1 = df_new[cols]
sw_df0 = df_old[cols]

# Set indices & merge
sw_df1.set_index(new_index, inplace='True')
sw_df0.set_index(new_index, inplace='True')

df = sw_df0.merge(sw_df1, how='outer', right_index=True, left_index=True, suffixes=('_old','_new'))
df

visits_old  \
group_site            KA_initiative endpoint_category            date                       
ABC Mouse             DDM           mobile-web                   2015-06-01  3.560085e+06   
                                                                 2015-07-01  2.979695e+06   
                                                                 2015-08-01  2.575315e+06   
                                                                 2015-09-01  2.133031e+06   
                                                                 2015-10-01  1.684011e+06   
                                                                 2015-11-01  1.423237e+06   
                                                                 2015-12-01  1.206644e+06   
                                                                 2016-01-01  1.884471e+06   
                                                                 2016-02-01  1.709895e+06   
                                                                 2016-03-01  1.626510e+06   
                                                                 2016-04-01  1.782128e+06   
                                                                 2016-05-01  2.503987e+06   
                                                                 2016-06-01  2.229293e+06   
                                                                 2016-07-01  2.773672e+06   
                                                                 2016-08-01  3.199835e+06   
                                                                 2016-09-01  3.157692e+06   
                                                                 2016-10-01  3.527233e+06   
                                                                 2016-11-01  2.673945e+06   
                                                                 2016-12-01  2.668970e+06   
                                                                 2017-01-01  4.544720e+06   
                                                                 2017-02-01  2.879631e+06   
                                                                 2017-03-01  3.266086e+06   
                                                                 2017-04-01  2.750863e+06   
                                                                 2017-05-01  2.935207e+06   
                                                                 2017-06-01  2.488018e+06   
                                                                 2017-07-01  2.535550e+06   
                                                                 2017-08-01           NaN   
                                    total-traffic-and-engagement 2015-06-01  8.616934e+06   
                                                                 2015-07-01  7.003751e+06   
                                                                 2015-08-01  5.500769e+06   
                                                                 2015-09-01  4.634524e+06   
                                                                 2015-10-01  3.930403e+06   
                                                                 2015-11-01  3.207837e+06   
                                                                 2015-12-01  2.683372e+06   
                                                                 2016-01-01  3.591827e+06   
                                                                 2016-02-01  3.099291e+06   
                                                                 2016-03-01  3.088023e+06   
                                                                 2016-04-01  3.252405e+06   
                                                                 2016-05-01  4.083152e+06   
                                                                 2016-06-01  3.713104e+06   
                                                                 2016-07-01  4.410297e+06   
                                                                 2016-08-01  4.7

This boolean table compares the data to highlight any inconsistencies:

In [4]:
# Sets conditions to find inconsistencies between updates
c1 = df["visits_old"] == df["visits_new"]
c2 = df["average_visit_duration_old"] == df["average_visit_duration_new"]

# DataFrame containing all inconsistencies
df = df.sort_index(axis=1)[-(c1|c2)]

At first glance it seems that **the hypothesis is validated** and that these _ex post_ changes affected many past data for many websites to the tune of about half of the data set:

In [5]:
pct_affected = df.shape[0] / df_new.shape[0]
print("{0:.2f}% of the rows show inconsistencies!".format(pct_affected * 100))

45.03% of the rows show inconsistencies!


It looks like the _ex post_ changes spanned the entire history of the dataset and every site:

In [6]:
# Summary lists
sites = df.index.levels[0]
dates = df.index.levels[3]

print('{} sites were affected:'.format(len(list(sites))))
sorted(sites)

111 sites were affected:


['ABC Mouse',
 'ABCya!',
 'Albert',
 'Amazon TenMarks',
 'Benchprep',
 'Better Lesson',
 'Blackboard',
 'Boundless',
 'BrainPOP',
 'Brainly',
 'BrightBytes',
 'Brilliant',
 'CK12',
 'ClassDojo',
 'Clever',
 'Code.org',
 'CodeHS',
 'Codecademy',
 'Coolmath.com',
 'Course Hero',
 'Coursera',
 'Descomplica',
 'Desmos',
 'Disney',
 'Dreambox',
 'Duolingo',
 'EdX',
 'Edgenuity',
 'Edmodo',
 'Education.com',
 'EngageNY',
 'ExploreLearning Reflex',
 'First in Math',
 'Formative',
 'Front Row',
 'Funbrain',
 'Geekie',
 'GiftedandTalented',
 'Google Classroom',
 'GreatMinds/Eurekamath',
 'IXL',
 'Illustrative Math',
 'Instructure Canvas',
 'K12',
 'KA (SimilarWeb)',
 'Kahoot!',
 'Kaplan',
 'Knewton',
 'LearnZillion',
 'Learning A-Z',
 'Lumosity',
 'Magoosh',
 'Mangahigh',
 'Manhattan Prep',
 'MasteryConnect',
 'Math2Me',
 'MathSpace.co',
 'Mathletics',
 'Mathway',
 'McGraw Hill ALEKS',
 'MeSalva',
 'Membean',
 'Memrise',
 'Mindspark',
 'MobyMax',
 'Moodle',
 'New Classrooms Teach to One: Math',

In [7]:
print('The following dates were affected:'.format(len(list(sites))))
sorted(dates)

The following dates were affected:


['2015-06-01',
 '2015-07-01',
 '2015-08-01',
 '2015-09-01',
 '2015-10-01',
 '2015-11-01',
 '2015-12-01',
 '2016-01-01',
 '2016-02-01',
 '2016-03-01',
 '2016-04-01',
 '2016-05-01',
 '2016-06-01',
 '2016-07-01',
 '2016-08-01',
 '2016-09-01',
 '2016-10-01',
 '2016-11-01',
 '2016-12-01',
 '2017-01-01',
 '2017-02-01',
 '2017-03-01',
 '2017-04-01',
 '2017-05-01',
 '2017-06-01',
 '2017-07-01',
 '2017-08-01']

In [8]:
# Reset indices to do aggregate functions
df.reset_index(inplace=True)
df.drop(["KA_initiative"], axis=1, inplace=True)

Below counts the inconsistent dates (each date has `visits` and `average_visit_duration` data points) by site and whether the data was mobile-web only, combined web, or desktop only.

In [33]:
# Desktop-only traffic issues
desktop_df = df.groupby(["group_site","endpoint_category"]).count()
desktop_df = desktop_df.loc[:,['date']]
desktop_df[desktop_df['date'] > 0]

date
group_site                        endpoint_category                 
ABC Mouse                         mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
ABCya!                            mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Albert                            mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Amazon TenMarks                   mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Benchprep                         mobile-web                       9
                                  total-traffic-and-engagement    16
                                  traffic-and-engagement           1
Better Lesson                     mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Blackboard                        mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Boundless                         mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
BrainPOP                          mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Brainly                           mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
BrightBytes                       mobile-web                       2
                                  total-traffic-and-engagement     5
                                  traffic-and-engagement           1
Brilliant                         mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
CK12                              mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
ClassDojo                         mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Clever                            mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Code.org                          mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
CodeHS                            mobile-web                      16
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Codecademy                        mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement           1
Coolmath.com                      mobile-web                      18
                                  total-traffic-and-engagement    18
                                  traffic-and-engagement      

It looked like most of the new data was around mobile-web (which impacts combined web since it's the aggregate of mobile-web + desktop), so below isolates just the changes to desktop-web data.

(Filtering by any instance greater than `1` since August 2017 will be new data for every endpoint.)

In [31]:
# Desktop-web-only traffic issues
desktop_df = df.groupby(["group_site","endpoint_category"]).count()
desktop_df = desktop_df.loc[(slice(None),['traffic-and-engagement']),['date']]
desktop_df[desktop_df['date'] > 1]

,,date
group_site,endpoint_category,
Front Row,traffic-and-engagement,18
Mindspark,traffic-and-engagement,18
Toppr,traffic-and-engagement,2


The first two are new additions since I ran the pipeline last. Toppr also relatively recently added.